In [166]:
import pandas as pd
import numpy as np
import lightgbm as lgb


# 1. Import data

In [167]:
training_set = pd.read_csv('data/training_set_VU_DM.csv')

# 2. Define relevance

In [168]:
training_set['relevance'] = training_set['click_bool'] + 4 * training_set['booking_bool']

# 3. Missing values

## 3a. Find percentage of missing values

In [169]:
missing_values = training_set.isna().mean().sort_values(ascending=False)
missing_values

comp1_rate_percent_diff        0.980954
comp6_rate_percent_diff        0.980604
comp1_rate                     0.975813
comp1_inv                      0.973871
comp4_rate_percent_diff        0.973563
gross_bookings_usd             0.972089
comp7_rate_percent_diff        0.972064
comp6_rate                     0.951565
visitor_hist_starrating        0.949204
visitor_hist_adr_usd           0.948977
comp6_inv                      0.947366
comp4_rate                     0.938008
comp7_rate                     0.936401
srch_query_affinity_score      0.935986
comp4_inv                      0.930690
comp7_inv                      0.928117
comp3_rate_percent_diff        0.904646
comp2_rate_percent_diff        0.887818
comp8_rate_percent_diff        0.876021
comp5_rate_percent_diff        0.830367
comp3_rate                     0.690565
comp3_inv                      0.667028
comp8_rate                     0.613449
comp8_inv                      0.599160
comp2_rate                     0.591664


## 3b. Missing values handling

- prop_review_score : encode if there is review or not (binary) or 0
- if comp_inv = 0 (exp and comp have availability) and comp_rate = 0 (exp and comp have same rate) then comp_rate_percent_diff = 0
- if comp_inv = 0 and comp_rate = NaN then comp_rate_percent_diff = 0
- if comp_inv = 0 (exp and comp have availability) and comp_rate = 1 (exp has lower rate than comp) then comp_rate_percent_diff = value
- if comp_inv = 1 (exp has availability and comp not) and comp_rate = 0 (exp and comp have same rate) then comp_rate_percent_diff = 0?
- if comp_inv = 1 (exp has availability and comp not) and comp_rate = -1 (exp has higher rate than comp) then comp_rate_percent_diff = -value
- if comp_inv = 1 and comp_rate = NaN then comp_rate_percent_diff = 0
- if comp_inv = NaN (no comp) and comp_rate = NaN (no comp) then comp_rate_percent_diff = comp_rate = comp_inv = 0
- gross_bookings_usd : drop
- srch_query_affinity_score : drop
- location_score2 : drop
- visitor_hist_adr_usd : drop
- orig_destination_distance : drop
- visitor_hist_starrating : drop

### Missing competitor data

In [170]:
training_set.loc[training_set.comp2_inv == -1,['srch_id', 'prop_id','comp2_inv','comp2_rate','comp2_rate_percent_diff']]

,srch_id,prop_id,comp2_inv,comp2_rate,comp2_rate_percent_diff
101,11,69993,-1.0,0.0,NaN
1123,81,138112,-1.0,NaN,NaN
2227,145,90958,-1.0,NaN,NaN
2811,180,74861,-1.0,NaN,NaN
2825,180,134534,-1.0,NaN,NaN
...,...,...,...,...,...
4956893,332667,75374,-1.0,NaN,NaN
4956991,332672,58952,-1.0,NaN,NaN
4957238,332693,48629,-1.0,NaN,NaN
4957481,332718,109431,-1.0,NaN,NaN


In [171]:
# Handle missing competitor data

for i in range(1,9):
    comp_inv_name = 'comp'+str(i)+'_inv'
    comp_rate_name = 'comp'+str(i)+'_rate'
    comp_rate_percent_diff_name = 'comp'+str(i)+'_rate_percent_diff'
    
    training_set[comp_inv_name] = training_set[comp_inv_name].fillna(0)
    training_set[comp_inv_name].replace(-1, 0, inplace=True)
    training_set[comp_rate_name] = training_set[comp_rate_name].fillna(0)
    training_set[comp_rate_percent_diff_name] = training_set[comp_rate_percent_diff_name].fillna(0) * training_set[comp_rate_name]
    
    training_set.drop(columns=comp_rate_name, inplace=True)
    

### Drop columns and set missing review score to 0

In [172]:
#Drop columns with too many missing values
training_set.drop(columns=['gross_bookings_usd',
                           'srch_query_affinity_score',
                           'visitor_hist_adr_usd',
                           'orig_destination_distance',
                           'visitor_hist_starrating',
                           'prop_location_score2'], inplace=True) 

# Set missing review scores to 0
training_set['prop_review_score'] = training_set['prop_review_score'].fillna(0)

## 3c. Recheck missing value percentage

In [173]:
training_set.isna().mean().sort_values(ascending=False)

srch_id                        0.0
random_bool                    0.0
comp1_rate_percent_diff        0.0
comp2_inv                      0.0
comp2_rate_percent_diff        0.0
comp3_inv                      0.0
comp3_rate_percent_diff        0.0
comp4_inv                      0.0
comp4_rate_percent_diff        0.0
comp5_inv                      0.0
comp5_rate_percent_diff        0.0
comp6_inv                      0.0
comp6_rate_percent_diff        0.0
comp7_inv                      0.0
comp7_rate_percent_diff        0.0
comp8_inv                      0.0
comp8_rate_percent_diff        0.0
click_bool                     0.0
booking_bool                   0.0
comp1_inv                      0.0
srch_saturday_night_bool       0.0
date_time                      0.0
srch_room_count                0.0
site_id                        0.0
visitor_location_country_id    0.0
prop_country_id                0.0
prop_id                        0.0
prop_starrating                0.0
prop_review_score   

# 4. Feature engineering

## 4a. Dates

In [174]:
# Convert date_time to datetime object
training_set['date_time'] = pd.to_datetime(training_set['date_time'])
training_set['month'] = training_set['date_time'].dt.month
training_set['day'] = training_set['date_time'].dt.day
training_set.drop(columns=['date_time'], inplace=True)

## 4b. Transform features

In [175]:
comp_vars = [i for i in training_set.columns if 'comp' in i]

epsilon = 1e-5
training_set['log_price_usd'] = np.log(training_set['price_usd'] + epsilon)
training_set.drop(columns=['price_usd'], inplace=True)

## 4c. Aggregate features

### By srch_id

In [176]:
srch_id_column_target = ['log_price_usd',
                         'prop_starrating',
                         'prop_location_score1',
                         'prop_review_score',
                         'prop_brand_bool'] + comp_vars


In [177]:
df_srch_id_grouped = training_set.groupby('srch_id')[srch_id_column_target].mean()
srch_id_grouped_columns = [i + '_MEAN | srch_id' for i in srch_id_column_target]
df_srch_id_grouped.columns = srch_id_grouped_columns

In [178]:
training_set = training_set.merge(df_srch_id_grouped, how='left', left_on='srch_id', right_index=True)

### By prop_id

In [179]:
prop_id_column_target = ['log_price_usd',
                         'prop_starrating',
                         'prop_brand_bool',
                         'promotion_flag'] + comp_vars

In [180]:
df_prop_id_grouped = training_set.groupby('prop_id')[prop_id_column_target].mean()
prop_id_grouped_columns = [i + '_MEAN | prop_id' for i in prop_id_column_target]
df_prop_id_grouped.columns = prop_id_grouped_columns

In [181]:
training_set = training_set.merge(df_prop_id_grouped, how='left', left_on='prop_id', right_index=True)

### By srch_destination_id

In [182]:
srch_destination_id_column_target = ['log_price_usd', 
                                     'prop_starrating',
                                     'prop_location_score1',
                                     'prop_review_score',
                                     'prop_brand_bool'] + comp_vars

In [183]:
df_srch_destination_id_grouped = training_set.groupby('srch_destination_id')[srch_destination_id_column_target].mean()
srch_destination_id_grouped_columns = [i + '_MEAN | srch_destination_id' for i in srch_destination_id_column_target]
df_srch_destination_id_grouped.columns = srch_destination_id_grouped_columns

In [184]:
training_set = training_set.merge(df_srch_destination_id_grouped, how='left', left_on='srch_destination_id', right_index=True)

In [185]:
training_set.describe()

,srch_id,site_id,visitor_location_country_id,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_log_historical_price,...,comp4_inv_MEAN | srch_destination_id,comp4_rate_percent_diff_MEAN | srch_destination_id,comp5_inv_MEAN | srch_destination_id,comp5_rate_percent_diff_MEAN | srch_destination_id,comp6_inv_MEAN | srch_destination_id,comp6_rate_percent_diff_MEAN | srch_destination_id,comp7_inv_MEAN | srch_destination_id,comp7_rate_percent_diff_MEAN | srch_destination_id,comp8_inv_MEAN | srch_destination_id,comp8_rate_percent_diff_MEAN | srch_destination_id
count,4.958347e+06,4.958347e+06,4.958347e+06,4.958347e+06,4.958347e+06,4.958347e+06,4.958347e+06,4.958347e+06,4.958347e+06,4.958347e+06,...,4.958347e+06,4.958347e+06,4.958347e+06,4.958347e+06,4.958347e+06,4.958347e+06,4.958347e+06,4.958347e+06,4.958347e+06,4.958347e+06
mean,1.663666e+05,9.953133e+00,1.753405e+02,1.739739e+02,7.007918e+04,3.180525e+00,3.772166e+00,6.346994e-01,2.872589e+00,4.317913e+00,...,7.612819e-03,1.164961e+00,2.994083e-02,1.149781e+00,4.561601e-03,1.440613e-01,6.844620e-03,2.708193e-01,1.026874e-02,2.426579e-01
std,9.611223e+04,7.646890e+00,6.591625e+01,6.834525e+01,4.060992e+04,1.051024e+00,1.059583e+00,4.815144e-01,1.531011e+00,1.834869e+00,...,2.439295e-02,2.323968e+01,2.696855e-02,6.939586e+00,1.311478e-02,4.957525e-01,1.935904e-02,1.253915e+00,1.186700e-02,3.114122e+00
min,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,-1.446364e+01,0.000000e+00,-4.738095e+01,0.000000e+00,-1.633333e+01,0.000000e+00,-9.500000e+00,0.000000e+00,-1.540000e+01
25%,8.293600e+04,5.000000e+00,1.000000e+02,1.000000e+02,3.501000e+04,3.000000e+00,3.500000e+00,0.000000e+00,1.790000e+00,4.450000e+00,...,0.000000e+00,0.000000e+00,1.190476e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-3.903542e-01
50%,1.665070e+05,5.000000e+00,2.190000e+02,2.190000e+02,6.963800e+04,3.000000e+00,4.000000e+00,1.000000e+00,2.770000e+00,4.910000e+00,...,0.000000e+00,0.000000e+00,2.608696e-02,3.197492e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,8.116883e-03,-1.003771e-02
75%,2.497240e+05,1.400000e+01,2.190000e+02,2.190000e+02,1.051680e+05,4.000000e+00,4.500000e+00,1.000000e+00,4.040000e+00,5.310000e+00,...,2.668090e-04,0.000000e+00,4.231528e-02,1.041214e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.555556e-02,5.233990e-03
max,3.327850e+05,3.400000e+01,2.310000e+02,2.300000e+02,1.408210e+05,5.000000e+00,5.000000e+00,1.000000e+00,6.980000e+00,6.210000e+00,...,9.090909e-01,1.493989e+03,1.000000e+00,8.198936e+02,3.333333e-01,2.321429e+01,7.380952e-01,3.051250e+02,5.882353e-01,1.057185e+02


In [186]:
training_set.loc[:,'comp2_inv'].min()

0.0

## 4d. Scale features

- star rating : min max scaler
- review score : min max scaler
- location score : min max scaler
- log price and log hist price : standard scaler
- length of stay : standard scaler
- booking window : standard scaler
- adults, children, room count : standard scaler
- comp features : standard scaler

In [197]:
# import standard scaler and minmax scaler
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# create standard scaler object
scaler = StandardScaler()

features_to_standardize = [i for i in training_set.columns if 'log_' in i] + \
                    [i for i in training_set.columns if 'count' in i and 'id' not in i] + \
                    [i for i in training_set.columns if 'comp' in i and 'rate_percent_diff' in i] + \
                    [i for i in training_set.columns if 'length_of_stay' in i] + \
                    [i for i in training_set.columns if 'booking_window' in i]

# create minmax scaler object
minmax_scaler = MinMaxScaler()

features_to_minmax = [i for i in training_set.columns if 'starrating' in i] + \
                [i for i in training_set.columns if 'review_score' in i] + \
                [i for i in training_set.columns if 'location_score' in i]
                
# fit scalers
scaler.fit(training_set[features_to_standardize])
minmax_scaler.fit(training_set[features_to_minmax])

# transform data
training_set[features_to_standardize] = scaler.transform(training_set[features_to_standardize])
training_set[features_to_minmax] = minmax_scaler.transform(training_set[features_to_minmax])


In [200]:
training_set.columns.values

array(['srch_id', 'site_id', 'visitor_location_country_id',
       'prop_country_id', 'prop_id', 'prop_starrating',
       'prop_review_score', 'prop_brand_bool', 'prop_location_score1',
       'prop_log_historical_price', 'position', 'promotion_flag',
       'srch_destination_id', 'srch_length_of_stay',
       'srch_booking_window', 'srch_adults_count', 'srch_children_count',
       'srch_room_count', 'srch_saturday_night_bool', 'random_bool',
       'comp1_inv', 'comp1_rate_percent_diff', 'comp2_inv',
       'comp2_rate_percent_diff', 'comp3_inv', 'comp3_rate_percent_diff',
       'comp4_inv', 'comp4_rate_percent_diff', 'comp5_inv',
       'comp5_rate_percent_diff', 'comp6_inv', 'comp6_rate_percent_diff',
       'comp7_inv', 'comp7_rate_percent_diff', 'comp8_inv',
       'comp8_rate_percent_diff', 'click_bool', 'booking_bool',
       'relevance', 'year', 'month', 'day', 'log_price_usd',
       'log_price_usd_MEAN | srch_id', 'prop_starrating_MEAN | srch_id',
       'prop_location_sco

NEXT STEP : 

- REMOVE TRAINING ONLY COLUMNS (AND MAYBE INCLUDE POSITION IN TEST SET?)
- SPLIT TRAINING AND VALIDATION SETS
- DEFINE NEEDS OF LIGHT GBM
    - CATEGORICAL FEATURES
    - GROUPS AND EVAL GROUPS
    - LABEL